In [1]:
import tensorflow as tf
import os
import numpy as np
from math import pi

### 2.2 Representing tensors###
Tensorflow uses the infamous __tensors__ to represent matrices of any dimensionality. (E.g. a 3D matrix can be represented
as a _rank 3_ tensor in tensorflow). <br>
Tensors cann be created from different sources such as _Python lists, numpy-arrays, tensorflow-constants, etc._ by using the: <br> `tf.convert_to_tensor([source])` <br>
function. `Notice that tf.constant()` also returns a Tensor object:

In [9]:
py_list = [[1.0, 2.0],
           [3.0, 4.0]]

numpy_array = np.array([[1.0, 2.0],
                        [3.0, 4.0]], dtype=np.float32)

tf_constant = tf.constant([[1.0, 2.0],
                           [3.0, 4.0]])

tensor1 = tf.convert_to_tensor(py_list, dtype=tf.float32)
tensor2 = tf.convert_to_tensor(numpy_array, dtype=tf.float32)
tensor3 = tf.convert_to_tensor(tf_constant, dtype=tf.float32)

print(type(py_list))
print(type(numpy_array))
print(type(tensor3))
print(type(tf_constant))
print("----")
print(tensor3)
print(tf_constant)

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
----
Tensor("Const_12:0", shape=(2, 2), dtype=float32)
Tensor("Const_12:0", shape=(2, 2), dtype=float32)


As you can see from the print statements above, each Tensor object has a unique __name__, __shape__, and __dtype__. Because we did not explicitly provide a name, tensorflow automatically generated names for us (e.g. "Const_12_0"). <br><br>
Tensorflow has a few convenient constructors for initialising frequently used tensors such as: <br>
- `tf.zeros(shape)`
- `tf.ones(shape)`

These can be used with standard numerical operations to create other numbers. For example, we can create a 500x500 tensor containing of 0.5s:

In [10]:
halves = tf.ones([500, 500]) * 0.5

### 2.3 Creating Operators ###
Tensorflow comes with its own operators which take tensors as their arguments. One of the simplest ones is the `tf.negative(tensor)` which takes a tensor as input and returns another tensor with every element of the input negated:


In [14]:
x = tf.constant([1.0,2.0])
x_neg = tf.negative(x)

Here is a list of other commonly used operators in tensorflow:
- `tf.add(x, y)`
- `tf.substract(x, y)`
- `tf.multiply(x, y)` (Note: element-wise)
- `tf.div(x, y)`
- `tf.floordiv(x, y)`
- `tf.truediv(x, y)` (Casts the arguments as float)
- `tf.mod(x, y)` Element wise remainder after dividing x by y
- `tf.pow(x, y)` Element wise power of x to y
- `tf.exp(x)`
- `tf.sqrt(x)`

<a href="https://www.tensorflow.org/api_guides/python/math_ops">Here</a> you can find the complete list of operators available in the documentation. Using the above operators, we can now write the gaussian pdf using tensors:

In [17]:
mean = 0.0
sigma = 1.0

norm_tensor = tf.exp(tf.negative(tf.pow(x - mean, 2.0)/(2.0 * tf.pow(sigma,2.0) ))) * (1.0 / (sigma * tf.sqrt(2.0 * pi) ))

### 2.4 Executing operators with sessions ###
Notice that until now, we have simply defined the behaviour of tensors - no calculations have yet been performed. For calculations to be carried out, a __session__ is required. To create a session we use the `tf.session()` function:

In [20]:
x = tf.constant([1., 2., 3.], dtype=tf.float32)
neg_x = tf.negative(x)

with tf.Session() as sess:
    result = sess.run(neg_x)

print(result)

[-1. -2. -3.]


Every Tensor object has an `eval()` function to evaluate the mathematical operations that defines its value. However, the `eval()` function requires defining a session object for the library to understand how best to make use of hardware. When we use `sess.run(...)` it is equivalent to invoking the Tensor's `eval()` function within the context of the created session.<br>

Sometimes, for presentation purposes it is more convenient to create an __interactive session__ where the session is implicit in any call to `eval()`. It is important to remember to `close()` the interactive mode to free up the taken resources:

In [21]:
sess = tf.InteractiveSession()

x = tf.constant([1., 2., 3.], dtype=tf.float32)
mean = 0.0
sigma = 1.0
norm_tensor = tf.exp(tf.negative(tf.pow(x - mean, 2.0)/(2.0 * tf.pow(sigma,2.0) ))) * (1.0 / (sigma * tf.sqrt(2.0 * pi) ))

result = norm_tensor.eval()
print(result)

sess.close()

[ 0.24197073  0.05399097  0.00443185]


#### Session configurations ####
Until now, both `tf.Session()` and `tf.InteractiveSession()` were called without any arguments. However, it is possible to pass a configuration object as an argument to achieve specific behaviour. In the example below, we tell TensorFlow to output info about which CPU/GPU devices are being used. This is useful, because TensorFlow determines the 'best' way automatically behind the scenes so we cannot be sure which device is being used.

In [3]:
# Note: The information is output in the command prompt - NOT in Jupyter
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_x)

print(result)

[-1. -2.]


A session does a lot more than "run the math". It can also take __placeholders__, __variables__, and __constants__ as input (although we have only seen constants until now). Here is a description of the main tensor types:
- __Placeholder__: A value that is unassigned but will be initialised by the session.
- __Variable__: A value that can change such as the weight values in a matrix.
- __Constant__: A value that does not change, such as the hyper-parameters/settings.

### 2.6 Using Variables ###
In order to build any machine learning model, we will need data to change so we will need to move from constants to variables. To create a variable we use the `tf.Variable()` function. Notice that we not only need to assign the variable but also initialise it using the `.initializer.run()` method of the tf.Variable class:

In [15]:
sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]

gt5 = tf.Variable(False)
gt5.initializer.run()

for elem in raw_data:
    tf.assign(gt5, elem > 5).eval()
    # updater = tf.assign(gt5, elem > 5)
    # updater.eval()
    # print("updater is: {}".format(updater.eval()))
    
    print("For {} gt5 is: {}".format(elem, gt5.eval()))

sess.close()

For 1.0 gt5 is: False
For 2.0 gt5 is: False
For 8.0 gt5 is: True
For -1.0 gt5 is: False
For 0.0 gt5 is: False
For 5.5 gt5 is: True
For 6.0 gt5 is: True
For 13 gt5 is: True


### 2.7 Saving and Loading Variables ###
Being able to save and load data makes debugging a lot easier. Also, you might want to use your trained weights after a week of training on a different computer or different session without having to retrain the same model. This would require saving the weight values to disk. <br>
TensorFlow provides this functionality

In [6]:
# Suppose we want to save results so that they are accessible without retraining the whole model.
# Tensorflow offers us a simple way to do this:

sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()
# tf.global_variables_initializer()
saver = tf.train.Saver()
 
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        print("SPIKE!")
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()
     
 
 
save_path = saver.save(sess, "./spikes.ckpt")
# print("spikes data saved in file: %s" % save_path)
 
sess.close()


SPIKE!
SPIKE!
SPIKE!


FailedPreconditionError: Attempting to use uninitialized value spikes
	 [[Node: save_4/SaveV2 = SaveV2[dtypes=[DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL, DT_BOOL], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_4/Const_0, save_4/SaveV2/tensor_names, save_4/SaveV2/shape_and_slices, spikes, spikes_1, spikes_2, spikes_3, spikes_4)]]

Caused by op 'save_4/SaveV2', defined at:
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernela

If you check in the notebook's directory, you will notice that the saver created a couple of files. To retrieve this data, we can use `restore`:

In [31]:
sess = tf.InteractiveSession()

spikes = tf.Variable([False]*8, name='spikes')
saver = tf.train.Saver()

saver.restore(sess, './spikes.ckpt.data-00000-of-00001')
print(spikes.eval())

sess.close()

INFO:tensorflow:Restoring parameters from ./spikes.ckpt.data-00000-of-00001


DataLossError: Unable to open table file .\spikes.ckpt.data-00000-of-00001: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[Node: save_14/RestoreV2_22 = RestoreV2[dtypes=[DT_BOOL], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_14/Const_0, save_14/RestoreV2_22/tensor_names, save_14/RestoreV2_22/shape_and_slices)]]
	 [[Node: save_14/Assign_12/_50 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_168_save_14/Assign_12", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save_14/RestoreV2_22', defined at:
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\mrfksiv\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _ru

#### Visualising data using TensorBoard ####
Tensorboard allows us to visualise the changing data of our models. This gives us an intuitive idea of how our model is performing.

In [5]:
raw_data = np.random.normal(10, 1, 100)
alpha = tf.constant(0.05)

curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.)
update_avg = alpha * curr_value + (1 - alpha) * prev_avg

# Create a summary for the average
avg_hist = tf.summary.scalar("running_average", update_avg)

# Create a summary for the values
value_hist = tf.summary.scalar("incoming_values", curr_value)

# Merge the summaries to make it easier to run together
merged = tf.summary.merge_all()

# Pass in the logs directory to the Filewriter
writer = tf.summary.FileWriter("./logs")

# Initialise all the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        
        # run the merged operation together with the update_avg operation in a single sess.run() call
        summary_str, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print(raw_data[i], curr_avg)
        
        # add the summary to the writer
        writer.add_summary(summary_str, i)

11.6151811339 0.580759
9.83940716172 1.04369
11.6862879759 1.57582
9.2771013891 1.96089
10.3455652754 2.38012
10.6513168123 2.79368
10.9147576486 3.19973
9.39740317908 3.50962
9.93459261762 3.83087
11.2074739475 4.1997
9.5969666126 4.46956
9.0748199315 4.69982
9.65823205199 4.94774
8.66748457147 5.13373
9.65410676722 5.35975
10.9395576186 5.63874
9.29321405383 5.82146
12.497267348 6.15525
10.6888929487 6.38193
10.0050707302 6.56309
9.78773703917 6.72432
10.8762098881 6.93192
9.69378606095 7.07001
8.23520342736 7.12827
12.0458722492 7.37415
8.57954938056 7.43442
8.82165297865 7.50378
9.55118880186 7.60615
8.49456050193 7.65057
9.29317686967 7.7327
11.4180343991 7.91697
10.164476975 8.02934
10.2096208141 8.13836
10.5063767637 8.25676
9.5408130226 8.32096
7.73017474275 8.29142
10.2404118828 8.38887
9.51319401378 8.44509
9.60997180392 8.50333
10.5160735914 8.60397
10.3581564212 8.69168
9.42071178072 8.72813
10.1008147698 8.79676
7.95661053516 8.75476
9.88954084391 8.8115
10.6126913843 8.90

After the model runs, enter the command `tensorboard --logdir=./logs` in the terminal/command prompt and then open a browser and navigate to <a>http://localhost:6006</a>. Welcome to TensorBoard!